# Test Project Cards ¶

In [1]:
import os
import sys
import yaml
import pickle
import glob
import copy

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

# Remote I/O and parameters

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
#root_dir = "D:/github/lfs-cleaning/travel-model-two-networks"
#input_dir = os.path.join(root_dir, 'data', 'processed', 'version_11')
output_dir = "C:/Wrangler Test"
card_dir = "M:\Application\Model Two\RTP2021\Project Cards\SON070004_101_MarinSonNarrows_Phase2"
lasso_dir = "C:/Users/bespin/Documents/GitHub/Lasso"

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-09-15 13:34:46, INFO: Lasso base directory set as: C:/Users/bespin/Documents/GitHub/Lasso
2021-09-15 13:34:46, INFO: Lasso base directory set as: C:/Users/bespin/Documents/GitHub/Lasso
2021-09-15 13:34:46, INFO: Lasso base directory set as: C:/Users/bespin/Documents/GitHub/Lasso
2021-09-15 13:34:46, INFO: Lasso base directory set as: C:/Users/bespin/Documents/GitHub/Lasso


# Read pickle

In [6]:
version_01_pickle_file_name = "C:/Wrangler Test/working_scenario_01/working_scenario_01.pickle"
v_01_scenario = pickle.load(open(version_01_pickle_file_name, 'rb'))

In [7]:
v_01_scenario.road_net.links_df.shape

(1634769, 31)

In [8]:
v_01_scenario.road_net.nodes_df.shape

(644475, 12)

In [9]:
v_01_scenario.road_net.shapes_df.shape

(869970, 7)

# Create scenario and apply projects

In [10]:
US101HOV_scenario = Scenario.create_scenario(
    base_scenario=v_01_scenario,
    card_directory = card_dir,
    validate_project_cards=False)

2021-09-15 13:37:55, INFO: Creating Scenario
2021-09-15 13:37:55, INFO: Creating Scenario


In [11]:
US101HOV_scenario.apply_all_projects()

2021-09-15 13:38:19, INFO: Applying US 101 Marin/Sonoma Narrows (Sonoma, Phase 2)
2021-09-15 13:38:19, INFO: Applying US 101 Marin/Sonoma Narrows (Sonoma, Phase 2)
2021-09-15 13:38:19, INFO: Applying US 101 Marin/Sonoma Narrows (Sonoma, Phase 2)
2021-09-15 13:38:19, INFO: Applying US 101 Marin/Sonoma Narrows (Sonoma, Phase 2)
2021-09-15 13:38:19, INFO: Applying Project to Roadway Network: US 101 Marin/Sonoma Narrows (Sonoma, Phase 2)
2021-09-15 13:38:19, INFO: Applying Project to Roadway Network: US 101 Marin/Sonoma Narrows (Sonoma, Phase 2)


c:\users\bespin\anaconda3\envs\my_lasso_environment\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [12]:
US101HOV_scenario.applied_projects

['US 101 Marin/Sonoma Narrows (Sonoma, Phase 2)']

## Make Travel Model Networks

In [13]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = US101HOV_scenario.road_net, 
    parameters = parameters)

2021-09-15 13:48:09, INFO: Lasso base directory set as: C:/Users/bespin/Documents/GitHub/Lasso
2021-09-15 13:48:09, INFO: Lasso base directory set as: C:/Users/bespin/Documents/GitHub/Lasso
2021-09-15 13:48:09, INFO: Lasso base directory set as: C:/Users/bespin/Documents/GitHub/Lasso
2021-09-15 13:48:09, INFO: Lasso base directory set as: C:/Users/bespin/Documents/GitHub/Lasso
2021-09-15 13:48:09, INFO: Filling nan for network from network wrangler
2021-09-15 13:48:09, INFO: Filling nan for network from network wrangler
2021-09-15 13:48:32, INFO: Converting variable type to mtc standard
2021-09-15 13:48:32, INFO: Converting variable type to mtc standard


In [14]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(1634769, 39)
(644475, 12)
(869970, 7)


In [15]:
model_net = mtc.roadway_standard_to_mtc_network(model_net, parameters)

2021-09-15 13:49:16, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-09-15 13:49:16, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-09-15 13:49:16, INFO: Creating managed lane network.
2021-09-15 13:49:16, INFO: Creating managed lane network.
2021-09-15 13:49:16, INFO: Creating network with duplicated managed lanes
2021-09-15 13:49:16, INFO: Creating network with duplicated managed lanes


c:\users\bespin\anaconda3\envs\my_lasso_environment\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\users\bespin\anaconda3\envs\my_lasso_environment\lib\site-packages\geopandas\geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
c:\users\bespin\anaconda3\envs\my_lasso_environment\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

2021-09-15 13:51:15, INFO: Determining cntype
2021-09-15 13:51:15, INFO: Determining cntype
2021-09-15 13:51:15, INFO: Lasso base directory set as: C:/Users/bespin/Documents/GitHub/Lasso
2021-09-15 13:51:15, INFO: Lasso base directory set as: C:/Users/bespin/Documents/GitHub/Lasso
2021-09-15 13:51:15, INFO: Lasso base directory set as: C:/Users/bespin/Documents/GitHub/Lasso
2021-09-15 13:51:15, INFO: Lasso base directory set as: C:/Users/bespin/Documents/GitHub/Lasso
2021-09-15 13:51:15, INFO: Calculating and adding roadway network variable: cntype
2021-09-15 13:51:15, INFO: Calculating and adding roadway network variable: cntype


AttributeError: 'Series' object has no attribute 'bus_only'

# Write to Disk

In [ ]:
model_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    link_output_variables = ["model_link_id", "A", "B", "geometry", "cntype", "lanes_AM", "assignable"],
    node_output_variables = ["model_node_id", "geometry", "farezone", "tap_id"],
    data_to_csv = False,
    data_to_dbf = True,
)

In [ ]:
model_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
    #drive_only = True
)

In [ ]:
%%capture
os.chdir(output_dir)
!make_cube.bat

In [ ]:
%%capture
!runtpp make_simple_roadway_network.s